<font color = 'red'  size = 4>**Unlike the original repo , I have used reversed tokens for source**</font>

## Import Libraries

In [1]:
# required for colab only
# commen this for local
#!pip3 install pickle5
#import pickle5 as pickle

     |████████████████████████████████| 256 kB 8.0 MB/s eta 0:00:01


In [11]:
# for local laptop uncomment this abd comment above cell
import pickle

In [ ]:
#!pip install -U spacy

In [ ]:
#!pip install -U swifter

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils import clip_grad_norm_, clip_grad_value_

import torchtext 
from torchtext.vocab import vocab
#from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np
from collections import Counter, OrderedDict

import random
import math
import time
import pandas as pd
from pathlib import Path
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

#import swifter

In [4]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
folder = Path('/home/harpreet/Insync/google_drive_harpreet/Research/NLP/pytorch-seq2seq')
#folder = Path('/content/drive/MyDrive/Research/NLP/pytorch-seq2seq')

In [3]:
torchtext.__version__, torch.__version__, torch.cuda.is_available(), spacy.__version__

('0.11.0', '1.10.0', True, '3.2.4')

# Set Seeds

In [4]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Load Tokenized Data
Next, we download and load the train, validation and test data. 

The dataset we'll be using is the [IIT Bombay English-Hindi Corpus](https://www.cfilt.iitb.ac.in/iitb_parallel/). 
Anoop Kunchukuttan, Pratik Mehta, Pushpak Bhattacharyya. The IIT Bombay English-Hindi Parallel Corpus. Language Resources and Evaluation Conference. 2018.

The datset can be  downloaded from huggingface library as well. 
We have downloaded and tokenized the data in Tutorial 1. We will now load the tokenized data.

In [8]:
## Required for colab only
'''
with open(folder/'df_train_hi_en', "rb") as fh:
  df_train = pickle.load(fh)
with open(folder/'df_test_hi_en', "rb") as fh:
  df_test = pickle.load(fh)
with open(folder/'df_valid_hi_en', "rb") as fh:
  df_valid = pickle.load(fh)  
'''

'\nwith open(folder/\'df_train_hi_en\', "rb") as fh:\n  df_train = pickle.load(fh)\nwith open(folder/\'df_test_hi_en\', "rb") as fh:\n  df_test = pickle.load(fh)\nwith open(folder/\'df_valid_hi_en\', "rb") as fh:\n  df_valid = pickle.load(fh)  \n'

In [5]:
# required for local (python 3.8)
df_train = pd.read_pickle(folder/'df_train_hi_en')
df_test = pd.read_pickle(folder/'df_test_hi_en')
df_valid = pd.read_pickle(folder/'df_valid_hi_en')

In [6]:
df_train.head()

,source_tokens_reversed,target_tokens,source_tokens
0,"[दें, लाभ, का, व्यायाम, पहुंचनीयता, को, अनुप्र...","[Give, your, application, an, accessibility, w...","[अपने, अनुप्रयोग, को, पहुंचनीयता, व्यायाम, का,..."
1,"[अन्वेषक, पहुंचनीयता, एक्सेर्साइसर]","[Accerciser, Accessibility, Explorer]","[एक्सेर्साइसर, पहुंचनीयता, अन्वेषक]"
2,"[खाका, प्लग-इन, डिफोल्ट, लिए, के, पटल, निचले]","[The, default, plugin, layout, for, the, botto...","[निचले, पटल, के, लिए, डिफोल्ट, प्लग-इन, खाका]"
3,"[खाका, प्लग-इन, डिफोल्ट, लिए, के, पटल, ऊपरी]","[The, default, plugin, layout, for, the, top, ...","[ऊपरी, पटल, के, लिए, डिफोल्ट, प्लग-इन, खाका]"
4,"[है, गया, किया, निष्क्रिय, से, रूप, डिफोल्ट, ज...","[A, list, of, plugins, that, are, disabled, by...","[उन, प्लग-इनों, की, सूची, जिन्हें, डिफोल्ट, रू..."


# Load Vocab

In [12]:
source_vocab = pickle.load(open(folder/'source_vocab_hi_en.pkl','rb'))
target_vocab = pickle.load(open(folder/'target_vocab_hi_en.pkl','rb'))

In [13]:
target_vocab['from']

55

In [14]:
len(source_vocab), len(target_vocab)

(6115, 6537)

# Create Dataset and Dataloader

In [15]:
class EngHindi(Dataset):
    
    '''
    Takes input as (X1, X2)
    X1 : pandas series for  source language
    X2 : pndas series for target language
    '''
    def __init__(self, X1, X2):
        self.X1 = X1
        self.X2 = X2
        
        
    def __len__(self):
        return len(self.X1)
    
    def __getitem__(self, indices):
        source_examples = self.X1.iloc[indices]  
        target_examples = self.X2.iloc[indices]
        return source_examples, target_examples    

In [16]:
trainset = EngHindi(df_train['source_tokens'], df_train['target_tokens'])
testset =  EngHindi(df_test['source_tokens'], df_test['target_tokens'])
validset = EngHindi(df_valid['source_tokens'], df_valid['target_tokens'])

In [17]:
trainset.__getitem__(0)

(['दें', 'लाभ', 'का', 'व्यायाम', 'पहुंचनीयता', 'को', 'अनुप्रयोग', 'अपने'],
 ['Give', 'your', 'application', 'an', 'accessibility', 'workout'])

In [18]:
len(trainset), len(testset), len(validset)

(1659083, 2507, 520)

In [19]:
len(trainset)*0.02

33181.66

In [20]:
# get subset of data
# We will be using only 100 images for nboth train and validation datasets
train_sample_size = int(len(trainset)*0.02)

# Getting n random indices
train_subset_indices = random.sample(range(0, len(trainset)), train_sample_size)

# Getting subset of dataset
train_subset = torch.utils.data.Subset(trainset, train_subset_indices)

In [21]:
print(train_subset.__getitem__(11))

(['।', 'होगा', 'सुनिश्चित', 'भविष्य', 'का', 'राष्ट्र', 'हमारे', 'और', 'बढ़ेगा', 'आधार', 'का', 'पीरामिड', 'नवाचार', 'हमारे', 'से', 'बोने', 'बीज', 'के', 'शक्ति', 'की', 'नवाचारों', 'और', 'विचारों', 'नए', 'में', 'बच्चों', 'स्कूली'], ['Seeding', 'the', 'power', 'of', 'ideas', 'and', 'innovation', 'in', 'schoolchildren', 'will', 'broaden', 'the', 'base', 'of', 'our', 'innovation', 'pyramid', 'and', 'secure', 'the', 'future', 'of', 'our', 'nation', '.'])


In [22]:
# transform text to indexes and append eos and bos
# finally convert to tensors
def text_transform(my_vocab, text):
    text_num = [my_vocab['<BOS>']] + [my_vocab[word] for word in text] + [my_vocab['<EOS>']]
    return torch.tensor(text_num)   

In [23]:
text = train_subset.__getitem__(13)[1]
print(text)

['Bad', 'current', 'tag', 'value', '.']


In [24]:
text_transform(target_vocab, text)

tensor([   1, 1032,   37,  672,  104,   33,    2])

In [25]:
def collate_batch(batch):
    source_list, target_list = [], []
    for source, target in batch:
        source_tensor = text_transform(source_vocab, source)
        target_tensor = text_transform(target_vocab, target)
        source_list.append(source_tensor)
        target_list.append(target_tensor)
        
    source_pad = pad_sequence(source_list, batch_first=False, padding_value= source_vocab['<PAD>'])
    target_pad = pad_sequence(target_list, batch_first=False, padding_value= target_vocab['<PAD>'])
    
    return source_pad, target_pad     

In [26]:
BATCH_SIZE = 128

train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle = True, collate_fn = collate_batch )
valid_loader = DataLoader(validset, batch_size=BATCH_SIZE, shuffle = False, collate_fn = collate_batch )
test_loader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle = False, collate_fn = collate_batch )

# 2a - Sequence to Sequence Learning with Neural Networks

## Introduction

Let's remind ourselves of the general encoder-decoder model.

![](assets/seq2seq1.png)

We use our encoder (green) over the embedded source sequence (yellow) to create a context vector (red). We then use that context vector with the decoder (blue) and a linear layer (purple) to generate the target sentence.

In the previous model, we used an multi-layered LSTM as the encoder and decoder.

![](assets/seq2seq4.png)

One downside of the previous model is that the decoder is trying to cram lots of information into the hidden states. Whilst decoding, the hidden state will need to contain information about the whole of the source sequence, as well as all of the tokens have been decoded so far. By alleviating some of this information compression, we can create a better model!

We'll also be using a GRU (Gated Recurrent Unit) instead of an LSTM (Long Short-Term Memory). Why? Mainly because that's what they did in the paper (this paper also introduced GRUs) and also because we used LSTMs last time. To understand how GRUs (and LSTMs) differ from standard RNNS, check out [this](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) link. Is a GRU better than an LSTM? [Research](https://arxiv.org/abs/1412.3555) has shown they're pretty much the same, and both are better than standard RNNs. 

## Building the Seq2Seq Model

### Encoder

The encoder is similar to the previous one, with the multi-layer LSTM swapped for a single-layer GRU. We also don't pass the dropout as an argument to the GRU as that dropout is used between each layer of a multi-layered RNN. As we only have a single layer, PyTorch will display a warning if we try and use pass a dropout value to it.

Another thing to note about the GRU is that it only requires and returns a hidden state, there is no cell state like in the LSTM.

$$\begin{align*}
h_t &= \text{GRU}(e(x_t), h_{t-1})\\
(h_t, c_t) &= \text{LSTM}(e(x_t), h_{t-1}, c_{t-1})\\
h_t &= \text{RNN}(e(x_t), h_{t-1})
\end{align*}$$

From the equations above, it looks like the RNN and the GRU are identical. Inside the GRU, however, is a number of *gating mechanisms* that control the information flow in to and out of the hidden state (similar to an LSTM). Again, for more info, check out [this](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) excellent post. 

The rest of the encoder should be very familar from the last tutorial, it takes in a sequence, $X = \{x_1, x_2, ... , x_T\}$, passes it through the embedding layer, recurrently calculates hidden states, $H = \{h_1, h_2, ..., h_T\}$, and returns a context vector (the final hidden state), $z=h_T$.

$$h_t = \text{EncoderGRU}(e(x_t), h_{t-1})$$

This is identical to the encoder of the general seq2seq model, with all the "magic" happening inside the GRU (green).

![](assets/seq2seq5.png)

In [36]:
class Encoder(nn.Module):
    
    def __init__(self, vocab_size, emb_dim, hidden_dim,
                 pad_idx, emb_drop_prob, num_layers, 
                 ):
      
      super().__init__()

      self.vocab_size = vocab_size
      self.emb_dim = emb_dim
      self.pad_idx= pad_idx
      self.emb_drop_prob=emb_drop_prob
      self.hidden_dim = hidden_dim
      self.num_layers = num_layers
      #self.rnn_drop_prob = rnn_drop_prob
      
      self.embedding = nn.Embedding(num_embeddings= self.vocab_size,
                                        embedding_dim=self.emb_dim,
                                        padding_idx=self.pad_idx)

      self.dropout = nn.Dropout(p = self.emb_drop_prob)

      self.gru_layer = nn.GRU(input_size=self.emb_dim,
                               hidden_size=self.hidden_dim,
                               num_layers=self.num_layers,
                               batch_first=False,
                               bidirectional=False,
                               #dropout = self.rnn_drop_prob
                                 )

      
        
    def forward(self, source_indices):
      # source_indices: [seq_len, batch_size]
      emb = self.embedding(source_indices) # shape : [seq_len, batch_size, emb_dim]
      emb_drop = self.dropout(emb) # shape : [seq_len, batch_size, emb_dim]
      
      
      output, hidden = self.gru_layer(emb_drop ) # h0 is optional
      # output : [seq_len, batch_size, directions * hidden_dim]
      # hidden : [directions* num_layers, batch_size, hidden_dim]

      return hidden


## Decoder

The decoder is where the implementation differs significantly from the previous model and we alleviate some of the information compression.

Instead of the GRU in the decoder taking just the embedded target token, $d(y_t)$ and the previous hidden state $s_{t-1}$ as inputs, it also takes the context vector $z$. 

$$s_t = \text{DecoderGRU}(d(y_t), s_{t-1}, z)$$

Note how this context vector, $z$, does not have a $t$ subscript, meaning we re-use the same context vector returned by the encoder for every time-step in the decoder. 

Before, we predicted the next token, $\hat{y}_{t+1}$, with the linear layer, $f$, only using the top-layer decoder hidden state at that time-step, $s_t$, as $\hat{y}_{t+1}=f(s_t^L)$. Now, we also pass the embedding of current token, $d(y_t)$ and the context vector, $z$ to the linear layer.

$$\hat{y}_{t+1} = f(d(y_t), s_t, z)$$

Thus, our decoder now looks something like this:

![](assets/seq2seq6.png)

Note, the initial hidden state, $s_0$, is still the context vector, $z$, so when generating the first token we are actually inputting two identical context vectors into the GRU.

How do these two changes reduce the information compression? Well, hypothetically the decoder hidden states, $s_t$, no longer need to contain information about the source sequence as it is always available as an input. Thus, it only needs to contain information about what tokens it has generated so far. The addition of $y_t$ to the linear layer also means this layer can directly see what the token is, without having to get this information from the hidden state. 

However, this hypothesis is just a hypothesis, it is impossible to determine how the model actually uses the information provided to it (don't listen to anyone that says differently). Nevertheless, it is a solid intuition and the results seem to indicate that this modifications are a good idea!

Within the implementation, we will pass $d(y_t)$ and $z$ to the GRU by concatenating them together, so the input dimensions to the GRU are now `emb_dim + hid_dim` (as context vector will be of size `hid_dim`). The linear layer will take $d(y_t), s_t$ and $z$ also by concatenating them together, hence the input dimensions are now `emb_dim + hid_dim*2`. We also don't pass a value of dropout to the GRU as it only uses a single layer.

`forward` now takes a `context` argument. Inside of `forward`, we concatenate $y_t$ and $z$ as `emb_con` before feeding to the GRU, and we concatenate $d(y_t)$, $s_t$ and $z$ together as `output` before feeding it through the linear layer to receive our predictions, $\hat{y}_{t+1}$.

In [37]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, 
                 pad_idx, emb_drop_prob, num_layers, 
                 ):
      
     
      super().__init__()

      self.vocab_size = vocab_size
      self.emb_dim = emb_dim
      self.pad_idx= pad_idx
      self.emb_drop_prob=emb_drop_prob
      self.hidden_dim = hidden_dim
      self.num_layers = num_layers
      #self.rnn_drop_prob = rnn_drop_prob
      

      self.embedding = nn.Embedding(num_embeddings= self.vocab_size,
                                        embedding_dim=self.emb_dim,
                                        padding_idx=self.pad_idx)

      self.dropout = nn.Dropout(p = self.emb_drop_prob) 

      self.gru_layer = nn.GRU(input_size=self.emb_dim + self.hidden_dim,
                               hidden_size=self.hidden_dim,
                               num_layers=self.num_layers,
                               batch_first=False,
                               bidirectional=False,
                               #dropout = self.rnn_drop_prob
                                 )
      
      self.linear= nn.Linear(in_features=self.hidden_dim + self.emb_dim + self.hidden_dim,
                                 out_features=self.vocab_size)

      
        
    def forward(self, input_token, hidden, hidden_src):

      # in decoder we pass one token at a time- seq_len is 1
      # shape of input_token : [batch_size]

      #input_token = input_token.unsqueeze(0) # [1, batch_size, emb_dim]
      #print(input_token.shape) 
            
      emb = self.dropout(self.embedding(input_token)) 
      # emb - [batch_size, emb_dim]
      
      # lstm layer needs input in the shape: [seq_len, batch_size, emb_dim]
      # we will add a redundant dimension to change  the shape of emb
      emb = emb.unsqueeze(0)
      
      #print(emb.shape)
      output, hidden = self.gru_layer(torch.cat((emb, hidden_src), dim = 2), hidden )

      # output - [seq_len, batch_size, directions * hidden_dim]
      # sequence length is always one for decoder as we pass one token at a time
      # we never use bidirectional for decoder as decoder cannot look ahead
      # hence shapes 
      # output: [1, batch_size, hidden_dim) 
      # hidden - [num_layers, batch_size, hidden_dim]
      
      # squeeze out the redundant dim from emd, output, enc_hidden to make it work for linear layer  
      output = output.squeeze(0)
      emb = emb.squeeze(0)
      hidden_src = hidden_src.squeeze(0)
        
      # output: [1, batch_size, hidden_dim) 
      prediction = self.linear(torch.cat((output, emb, hidden_src), dim =1))
      # prediction: [batch_size, decoder_vocab_size] 
      # each word is projected to vocab size
      
      return prediction, hidden

## Seq2Seq Model

Putting the encoder and decoder together, we get:

![](assets/seq2seq7.png)

Again, in this implementation we need to ensure the hidden dimensions in both the encoder and the decoder are the same.

Briefly going over all of the steps:
- the `outputs` tensor is created to hold all predictions, $\hat{Y}$
- the source sequence, $X$, is fed into the encoder to receive a `context` vector
- the initial decoder hidden state is set to be the `context` vector, $s_0 = z = h_T$
- we use a batch of `<sos>` tokens as the first `input`, $y_1$
- we then decode within a loop:
  - inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and the context vector, $z$, into the decoder
  - receiving a prediction, $\hat{y}_{t+1}$, and a new hidden state, $s_t$
  - we then decide if we are going to teacher force or not, setting the next input as appropriate (either the ground truth next token in the target sequence or the highest predicted next token)

In [38]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device ):

      super().__init__()

      self.decoder = decoder
      self.encoder = encoder
      self.device = device
      

      assert self.decoder.hidden_dim == self.encoder.hidden_dim, \
      'The hidden_dim for encoder and decoder should be equal'

      assert self.decoder.num_layers == self.encoder.num_layers, \
      ' The number of layers in encoder and decoder should be same'


        
    def forward(self, source_indices, target_indices, teacher_enf_ratio ):
      # source_indices, target_indices # [seq_len, batch_size]
      
      

      seq_len = target_indices.shape[0]
      batch_size =  target_indices.shape[1]
      vocab_size = self.decoder.vocab_size

      predictions = torch.zeros((seq_len, batch_size, vocab_size)).to(self.device) # [tar_len, batch_size, target_vocab_size]

      hidden_src = self.encoder(source_indices) # [directions * num_layer, batch_size, hidden_dim]
    
      input_token = target_indices[0,:] # [batch_size]
      
      hidden = hidden_src

      # we will not update the predictions corresponding to first token -<BOS> 

      for i in range(1, len(target_indices)):
        prediction, hidden = self.decoder(input_token, hidden, hidden_src) # prediction: [batch_size, decoder_vocab_size] 

        # update predictions
        predictions[i] = prediction

        teacher_force = torch.rand(1) < teacher_enf_ratio

        if teacher_force:
          input_token = target_indices[i,:]
        else:
          input_token = torch.argmax(prediction, dim =1) # batch_size

      return predictions # [tar_len, batch_size, target_vocab_size]   
    


# Training the Seq2Seq Model

The rest of this tutorial is very similar to the previous one. 

We initialise our encoder, decoder and seq2seq model (placing it on the GPU if we have one). As before, the embedding dimensions and the amount of dropout used can be different between the encoder and the decoder, but the hidden dimensions must remain the same.

In [61]:
# list of hyperparameters for encoder, decoder, model
ENC_VOCAB_SIZE = len(source_vocab)
ENC_EMB = 128
HID_DIM = 256
ENC_PAD_IDX = source_vocab['<PAD>']
ENC_EMB_DROP_PROB = 0.5
NUM_LAYERS = 1
#ENC_RNN_DROP_PROB = 0.5

DEC_VOCAB_SIZE = len(target_vocab)
DEC_EMB = 256
DEC_PAD_IDX = target_vocab['<PAD>']
DEC_EMB_DROP_PROB = 0.5
#DEC_RNN_DROP_PROB = 0.5

LEARNING_RATE = 0.001

enc = Encoder(vocab_size=ENC_VOCAB_SIZE , emb_dim =ENC_EMB, hidden_dim=HID_DIM,
              pad_idx=ENC_PAD_IDX, emb_drop_prob=ENC_EMB_DROP_PROB, num_layers=NUM_LAYERS, 
              )

dec = Decoder(vocab_size=DEC_VOCAB_SIZE, emb_dim=DEC_EMB, hidden_dim=HID_DIM, 
              pad_idx=DEC_PAD_IDX, emb_drop_prob=DEC_EMB_DROP_PROB, num_layers=NUM_LAYERS, 
              )

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

model = Seq2Seq(encoder=enc, decoder=dec, device=device )
model.to(device)

cuda


Seq2Seq(
  (decoder): Decoder(
    (embedding): Embedding(6537, 256, padding_idx=3)
    (dropout): Dropout(p=0.5, inplace=False)
    (gru_layer): GRU(512, 256)
    (linear): Linear(in_features=768, out_features=6537, bias=True)
  )
  (encoder): Encoder(
    (embedding): Embedding(6115, 128, padding_idx=3)
    (dropout): Dropout(p=0.5, inplace=False)
    (gru_layer): GRU(128, 256)
  )
)

Next, we initialize our parameters. The paper states the parameters are initialized from a normal distribution with a mean of 0 and a standard deviation of 0.01, i.e. $\mathcal{N}(0, 0.01)$. 

It also states we should initialize the recurrent parameters to a special initialization, however to keep things simple we'll also initialize them to $\mathcal{N}(0, 0.01)$.

In [62]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, 0, 0.01)
        
model.apply(init_weights)

Seq2Seq(
  (decoder): Decoder(
    (embedding): Embedding(6537, 256, padding_idx=3)
    (dropout): Dropout(p=0.5, inplace=False)
    (gru_layer): GRU(512, 256)
    (linear): Linear(in_features=768, out_features=6537, bias=True)
  )
  (encoder): Encoder(
    (embedding): Embedding(6115, 128, padding_idx=3)
    (dropout): Dropout(p=0.5, inplace=False)
    (gru_layer): GRU(128, 256)
  )
)

We also define a function that will calculate the number of trainable parameters in the model.

Even though we only have a single layer RNN for our encoder and decoder we actually have **more** parameters  than the last model. This is due to the increased size of the inputs to the GRU and the linear layer. However, it is not a significant amount of parameters and causes a minimal amount of increase in training time (~3 seconds per epoch extra).

In [63]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 8,370,953 trainable parameters


We define our optimizer, which we use to update our parameters in the training loop. Check out [this](http://ruder.io/optimizing-gradient-descent/) post for information about different optimizers. Here, we'll use Adam.

In [64]:
optimizer = optim.Adam(model.parameters())

Next, we define our loss function. The `CrossEntropyLoss` function calculates both the log softmax as well as the negative log-likelihood of our predictions. 

Our loss function calculates the average loss per token, however by passing the index of the `<pad>` token as the `ignore_index` argument we ignore the loss whenever the target token is a padding token. 

In [65]:
criterion = nn.CrossEntropyLoss(ignore_index=DEC_PAD_IDX)

Next, we'll define our training loop. 

First, we'll set the model into "training mode" with `model.train()`. This will turn on dropout (and batch normalization, which we aren't using) and then iterate through our data iterator.

As stated before, our decoder loop starts at 1, not 0. This means the 0th element of our `outputs` tensor remains all zeros. So our `trg` and `outputs` look something like:

$$\begin{align*}
\text{trg} = [<sos>, &y_1, y_2, y_3, <eos>]\\
\text{outputs} = [0, &\hat{y}_1, \hat{y}_2, \hat{y}_3, <eos>]
\end{align*}$$

Here, when we calculate the loss, we cut off the first element of each tensor to get:

$$\begin{align*}
\text{trg} = [&y_1, y_2, y_3, <eos>]\\
\text{outputs} = [&\hat{y}_1, \hat{y}_2, \hat{y}_3, <eos>]
\end{align*}$$

At each iteration:
- get the source and target sentences from the batch, $X$ and $Y$
- zero the gradients calculated from the last batch
- feed the source and target into the model to get the output, $\hat{Y}$
- as the loss function only works on 2d inputs with 1d targets we need to flatten each of them with `.view`
    - we slice off the first column of the output and target tensors as mentioned above
- calculate the gradients with `loss.backward()`
- clip the gradients to prevent them from exploding (a common issue in RNNs)
- update the parameters of our model by doing an optimizer step
- sum the loss value to a running total

Finally, we return the loss that is averaged over all batches.

In [66]:
def train(model, iterator, optimizer, criterion, clip, teacher_enf_ratio):

  model.train()
  epoch_loss = 0

  for src , tgt in iterator:
    src = src.to(device)
    tgt = tgt.to(device)
    

    # get predictions
    logits = model(src, tgt, teacher_enf_ratio)

    tgt = tgt[1:,:] # first seq corresponds to token '<BOS>'
    logits = logits[1:, :]

    num_classes  = logits.shape[-1]
    logits = logits.view(-1, num_classes) # [(trg_seq_length-1) * batch_size, num_classes]
    tgt = tgt.view(-1) # [(trg_seq_length-1) * batch_size]

    # set gradients to zero to avoid gradient accumulation from previous iterations
    optimizer.zero_grad

    # calculate loss
    #with torch.cuda.amp.autocast():
    loss = criterion(logits, tgt)

    # calculate gradients
    loss.backward()
    #scaler.scale(loss).backward()

    # clip gradients
    clip_grad_norm_(model.parameters(), clip)

    # update parameters
    optimizer.step()
    
    #scaler.step(optimizer)
    #scaler.update()
    
    epoch_loss+= loss.item()

  return epoch_loss/len(iterator)   

Our evaluation loop is similar to our training loop, however as we aren't updating any parameters we don't need to pass an optimizer or a clip value.

We must remember to set the model to evaluation mode with `model.eval()`. This will turn off dropout (and batch normalization, if used).

We use the `with torch.no_grad()` block to ensure no gradients are calculated within the block. This reduces memory consumption and speeds things up. 

The iteration loop is similar (without the parameter updates), however we must ensure we turn teacher forcing off for evaluation. This will cause the model to only use it's own predictions to make further predictions within a sentence, which mirrors how it would be used in deployment.

In [67]:
def evaluate(model, iterator, criterion, teacher_enf_ratio):
    model.eval()
    epoch_loss = 0
  

    with torch.no_grad():
      for src , tgt in iterator:

        src = src.to(device) # [src_seq_len, batch_size]

        tgt = tgt.to(device) # [tgt_seq_len, batch_size]

        # get predictions
        logits = model(src, tgt, teacher_enf_ratio) # [trg_seq_length, batch_size, tgt_vocab_size]

        tgt = tgt[1:,:] # first seq corresponds to token '<BOS>'
        logits = logits[1:, :]

        num_classes  = logits.shape[-1]
        logits = logits.view(-1, num_classes) # [(trg_seq_length-1) * batch_size, num_classes]
        tgt = tgt.view(-1) # [(trg_seq_length-1) * batch_size]

        # calculate loss
        loss = criterion(logits, tgt)

        epoch_loss+= loss.item()

    return epoch_loss/len(iterator)  

Next, we'll create a function that we'll use to tell us how long an epoch takes.

In [68]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

We can finally start training our model!

At each epoch, we'll be checking if our model has achieved the best validation loss so far. If it has, we'll update our best validation loss and save the parameters of our model (called `state_dict` in PyTorch). Then, when we come to test our model, we'll use the saved parameters used to achieve the best validation loss. 

We'll be printing out both the loss and the perplexity at each epoch. It is easier to see a change in perplexity than a change in loss as the numbers are much bigger.

In [69]:
import  gc
gc.collect()
torch.cuda.empty_cache()

In [70]:
N_EPOCHS = 10
CLIP = 1
TRAIN_TEACHER_ENF_RATIO = 0.5
VALID_TEACHER_ENF_RATIO = 0 # turn off teacher enforcing for evaluation

min_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_loader, optimizer, criterion, CLIP, TRAIN_TEACHER_ENF_RATIO)
    valid_loss = evaluate(model, valid_loader, criterion, VALID_TEACHER_ENF_RATIO)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    # save the model corresponding to min vlaid loss
    if valid_loss<min_valid_loss:
      min_valid_loss = valid_loss
      torch.save(model.state_dict(), folder/'2b_en_hi.pt')
    
   
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 1m 47s
	Train Loss: 5.918 | Train PPL: 371.627
	 Val. Loss: 5.873 |  Val. PPL: 355.267
Epoch: 02 | Time: 1m 48s
	Train Loss: 5.444 | Train PPL: 231.302
	 Val. Loss: 5.871 |  Val. PPL: 354.501
Epoch: 03 | Time: 1m 47s
	Train Loss: 5.267 | Train PPL: 193.831
	 Val. Loss: 5.815 |  Val. PPL: 335.439
Epoch: 04 | Time: 1m 48s
	Train Loss: 5.129 | Train PPL: 168.812
	 Val. Loss: 5.786 |  Val. PPL: 325.801
Epoch: 05 | Time: 1m 48s
	Train Loss: 4.946 | Train PPL: 140.628
	 Val. Loss: 5.700 |  Val. PPL: 298.742
Epoch: 06 | Time: 1m 47s
	Train Loss: 4.812 | Train PPL: 122.964
	 Val. Loss: 5.618 |  Val. PPL: 275.373
Epoch: 07 | Time: 1m 46s
	Train Loss: 4.672 | Train PPL: 106.947
	 Val. Loss: 5.558 |  Val. PPL: 259.415
Epoch: 08 | Time: 1m 48s
	Train Loss: 4.513 | Train PPL:  91.223
	 Val. Loss: 5.512 |  Val. PPL: 247.537
Epoch: 09 | Time: 1m 47s
	Train Loss: 4.370 | Train PPL:  79.035
	 Val. Loss: 5.487 |  Val. PPL: 241.552
Epoch: 10 | Time: 1m 46s
	Train Loss: 4.245 | Train PPL

We'll load the parameters (`state_dict`) that gave our model the best validation loss and run it the model on the test set.

In [71]:
# load the saved model and get test loss
model.load_state_dict(torch.load(folder/'2b_en_hi.pt'))
test_loss = evaluate(model, test_loader, criterion, 0)
print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 5.491 | Test PPL: 242.552 |


In the following notebook we'll implement a model that achieves improved test perplexity, but only uses a single layer in the encoder and the decoder.